## Model

Asumsikan bahwa setiap item input xi tidak lagi hanya berupa satu nilai, tetapi vector k yang terdiri dari xil,...,xik.

The multile regression berasumsi bahwa:
```
yi = α+β₁χɪʟ+...+βkχɪʟ
```

Dalam model multiple regression, vector parameter biasanya disebut beta.

Jika suku konstanta juga disertakan, kita capai dengan menambahkan column untuk data kita:

* beta = [alpha, beta_1, ... , beta_k]
* x_i = [1, x_i1, ... , x_ik]

In [23]:
# Kemudian model kita hanya:

from typing import List

Vector = List[float]

def dot(v: Vector, w: Vector) -> float:
    """Menghitung dot product dari dua vektor."""
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

def predict(x: Vector, beta: Vector) -> float:
    """Asumsikan bahwa element pertama dari x adalah 1"""
    return dot(x, beta)

Ini untuk permasalah khusus, independen variabel x kita list dari vector, terlihat seperti ini:
[
    1,      # constant term
    49,     # number of friends
    4,      # work hours per day
    0       # doesn't have phD
]

### Further Assumptions of the Least Squares Model

Ada beberapa asumsi lebih lanjut kebutuhan dari model ini (solusi) yang masuk akal:
1. Comluns dari x adalah `Linearly Independent`, bahwa tidak ada cara untuk menulis siapapun sebagai **weighted sum** dari beberapa yang lain. Jika asumsi ini gagal, ini mungkin to memperkirakan beta. untuk melihat kasus ekstrim ini, bayangkan kita memiliki bidang tambahan `num_acquaintances` pada data kita bahwa untuk setiap user tepat sama dengan `num_frieds`.

Jika asumsi ini tidak benar, β tidak dapat diestimasi.

2. Setiap elemen dalam x tidak bergantung pada suku galat ε.
Jika asumsi ini tidak benar, galat sistematis akan terjadi saat menaksir β.

### Sesuai model 


In [24]:
from typing import List

def error(x: Vector, y: float, beta: Vector) -> float:
    """Menghitung error dari prediksi."""
    return predict(x, beta) - y

def squared_error(x: Vector, y: float, beta: Vector) -> float:
    """Menghitung squared error dari prediksi."""
    return error(x, y, beta) ** 2

In [25]:
# Pengujian Model

x = [1, 2, 3]  # Contoh vektor input
y = 30 # Contoh nilai target
beta = [4, 4, 4]  # Contoh koefisien model
print("Prediksi:", predict(x, beta))  # Output: 40.0

assert error(x, y, beta) == -6
assert squared_error(x, y, beta) == 36
# Model ini mengasumsikan bahwa kita memiliki vektor input `x` yang berisi fitur-fitur

Prediksi: 24


In [26]:
# easy to compute the gradient

def sqerror_gradient(x: Vector, y: float, beta: Vector) -> Vector:
    """Menghitung gradien dari squared error."""
    e = error(x, y, beta)
    return [2 * e * x_i for x_i in x]

assert sqerror_gradient(x, y, beta) == [-12, -24, -36]
# yang sudah ditambahkan bias (elemen pertama adalah 1).

Mencari beta yang optimal menggunakan gradient descent:
1. menulis `least_squares_fit` function dapat berjalan dengan dataset apapun

In [27]:
import random
import tqdm

from typing import List

Vector = List[float]

def add(v: Vector, w: Vector) -> Vector:
    """Menambahkan dua vektor."""
    return [v_i + w_i for v_i, w_i in zip(v, w)]


def vector_sum(vectors: List[Vector]) -> Vector:
    """Menjumlahkan daftar vektor."""
    assert vectors, "Daftar vektor tidak boleh kosong"

    # Pastikan semua vektor memiliki panjang yang sama
    num_length = len((vectors[0]))
    assert all(len(v) == num_length for v in vectors), "Semua vektor harus memiliki panjang yang sama"

    return [sum(vector[i] for vector in vectors) for i in range(num_length)]

def scalar_multiply(c: float, v: Vector) -> Vector:
    """Mengalikan vektor dengan skalar."""
    return [c * v_i for v_i in v]

def vector_mean(vectors: List[Vector]) -> Vector:
    """Menghitung rata-rata dari daftar vektor."""
    n = len(vectors)
    return scalar_multiply(1/n, vector_sum(vectors))

def gradient_step(v: Vector, gradient: Vector, step_size: float) -> Vector:
    """Melangkah dalam arah gradien."""
    return add(v, scalar_multiply(step_size, gradient))

In [31]:
def least_squares_fit(xs: List[Vector], ys: List[float], learning_rate: float = 0.001, num_steps: int = 1000, batch_size: int = 1) -> Vector:
    """
    Mencari beta bahwa meminimalkan squared error pada data (xs, ys).
    assumsikan model y = dot(x, beta)
    """
    # Mulai dengan random gues
    guess = [random.random() for _ in xs[0]]

    for _ in tqdm.trange(num_steps, desc="least squares fit"):
        for start in range(0, len(xs), batch_size):
            batch_xs = xs[start:start + batch_size]
            batch_ys = ys[start:start + batch_size]

            gradient = vector_mean([sqerror_gradient(x, y, guess) for x, y in zip(batch_xs, batch_ys)])
            guess = gradient_step(guess, gradient, -learning_rate)

    return guess

### Terapkan ke data

In [32]:
inputs = [[1,49,4,0],[1,41,9,0],[1,40,8,0],[1,25,6,0],[1,21,1,0],[1,21,0,0],[1,19,3,0],[1,19,0,0],[1,18,9,0],[1,18,8,0],[1,16,4,0],[1,15,3,0],[1,15,0,0],[1,15,2,0],[1,15,7,0],[1,14,0,0],[1,14,1,0],[1,13,1,0],[1,13,7,0],[1,13,4,0],[1,13,2,0],[1,12,5,0],[1,12,0,0],[1,11,9,0],[1,10,9,0],[1,10,1,0],[1,10,1,0],[1,10,7,0],[1,10,9,0],[1,10,1,0],[1,10,6,0],[1,10,6,0],[1,10,8,0],[1,10,10,0],[1,10,6,0],[1,10,0,0],[1,10,5,0],[1,10,3,0],[1,10,4,0],[1,9,9,0],[1,9,9,0],[1,9,0,0],[1,9,0,0],[1,9,6,0],[1,9,10,0],[1,9,8,0],[1,9,5,0],[1,9,2,0],[1,9,9,0],[1,9,10,0],[1,9,7,0],[1,9,2,0],[1,9,0,0],[1,9,4,0],[1,9,6,0],[1,9,4,0],[1,9,7,0],[1,8,3,0],[1,8,2,0],[1,8,4,0],[1,8,9,0],[1,8,2,0],[1,8,3,0],[1,8,5,0],[1,8,8,0],[1,8,0,0],[1,8,9,0],[1,8,10,0],[1,8,5,0],[1,8,5,0],[1,7,5,0],[1,7,5,0],[1,7,0,0],[1,7,2,0],[1,7,8,0],[1,7,10,0],[1,7,5,0],[1,7,3,0],[1,7,3,0],[1,7,6,0],[1,7,7,0],[1,7,7,0],[1,7,9,0],[1,7,3,0],[1,7,8,0],[1,6,4,0],[1,6,6,0],[1,6,4,0],[1,6,9,0],[1,6,0,0],[1,6,1,0],[1,6,4,0],[1,6,1,0],[1,6,0,0],[1,6,7,0],[1,6,0,0],[1,6,8,0],[1,6,4,0],[1,6,2,1],[1,6,1,1],[1,6,3,1],[1,6,6,1],[1,6,4,1],[1,6,4,1],[1,6,1,1],[1,6,3,1],[1,6,4,1],[1,5,1,1],[1,5,9,1],[1,5,4,1],[1,5,6,1],[1,5,4,1],[1,5,4,1],[1,5,10,1],[1,5,5,1],[1,5,2,1],[1,5,4,1],[1,5,4,1],[1,5,9,1],[1,5,3,1],[1,5,10,1],[1,5,2,1],[1,5,2,1],[1,5,9,1],[1,4,8,1],[1,4,6,1],[1,4,0,1],[1,4,10,1],[1,4,5,1],[1,4,10,1],[1,4,9,1],[1,4,1,1],[1,4,4,1],[1,4,4,1],[1,4,0,1],[1,4,3,1],[1,4,1,1],[1,4,3,1],[1,4,2,1],[1,4,4,1],[1,4,4,1],[1,4,8,1],[1,4,2,1],[1,4,4,1],[1,3,2,1],[1,3,6,1],[1,3,4,1],[1,3,7,1],[1,3,4,1],[1,3,1,1],[1,3,10,1],[1,3,3,1],[1,3,4,1],[1,3,7,1],[1,3,5,1],[1,3,6,1],[1,3,1,1],[1,3,6,1],[1,3,10,1],[1,3,2,1],[1,3,4,1],[1,3,2,1],[1,3,1,1],[1,3,5,1],[1,2,4,1],[1,2,2,1],[1,2,8,1],[1,2,3,1],[1,2,1,1],[1,2,9,1],[1,2,10,1],[1,2,9,1],[1,2,4,1],[1,2,5,1],[1,2,0,1],[1,2,9,1],[1,2,9,1],[1,2,0,1],[1,2,1,1],[1,2,1,1],[1,2,4,1],[1,1,0,1],[1,1,2,1],[1,1,2,1],[1,1,5,1],[1,1,3,1],[1,1,10,1],[1,1,6,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,4,1],[1,1,9,1],[1,1,9,1],[1,1,4,1],[1,1,2,1],[1,1,9,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,1,1],[1,1,1,1],[1,1,5,1]]
daily_minutes_good = [68.77,51.25,52.08,38.36,44.54,57.13,51.4,41.42,31.22,34.76,54.01,38.79,47.59,49.1,27.66,41.03,36.73,48.65,28.12,46.62,35.57,32.98,35,26.07,23.77,39.73,40.57,31.65,31.21,36.32,20.45,21.93,26.02,27.34,23.49,46.94,30.5,33.8,24.23,21.4,27.94,32.24,40.57,25.07,19.42,22.39,18.42,46.96,23.72,26.41,26.97,36.76,40.32,35.02,29.47,30.2,31,38.11,38.18,36.31,21.03,30.86,36.07,28.66,29.08,37.28,15.28,24.17,22.31,30.17,25.53,19.85,35.37,44.6,17.23,13.47,26.33,35.02,32.09,24.81,19.33,28.77,24.26,31.98,25.73,24.86,16.28,34.51,15.23,39.72,40.8,26.06,35.76,34.76,16.13,44.04,18.03,19.65,32.62,35.59,39.43,14.18,35.24,40.13,41.82,35.45,36.07,43.67,24.61,20.9,21.9,18.79,27.61,27.21,26.61,29.77,20.59,27.53,13.82,33.2,25,33.1,36.65,18.63,14.87,22.2,36.81,25.53,24.62,26.25,18.21,28.08,19.42,29.79,32.8,35.99,28.32,27.79,35.88,29.06,36.28,14.1,36.63,37.49,26.9,18.58,38.48,24.48,18.95,33.55,14.24,29.04,32.51,25.63,22.22,19,32.73,15.16,13.9,27.2,32.01,29.27,33,13.74,20.42,27.32,18.23,35.35,28.48,9.08,24.62,20.12,35.26,19.92,31.02,16.49,12.16,30.7,31.22,34.65,13.13,27.51,33.2,31.57,14.1,33.42,17.44,10.12,24.42,9.82,23.39,30.93,15.03,21.67,31.09,33.29,22.61,26.89,23.48,8.38,27.81,32.35,23.84]

In [33]:
import numpy as np

random.seed(0)

# I used trial and error to choose num_iters and step_size
# This will run for a while, so you might want to reduce num_iters
learning_rate = 0.001

beta = least_squares_fit(inputs, daily_minutes_good, learning_rate, 5000, 25)
assert 30.50 < beta[0] < 30.70 # Constant term
assert 0.96 < beta[1] < 1.00 # num friends
assert -1.89 < beta[2] < -1.85 # work hours per day
assert 0.91 < beta[3] < 0.94 # has Phd

least squares fit:   0%|          | 0/5000 [00:00<?, ?it/s]

least squares fit: 100%|██████████| 5000/5000 [00:01<00:00, 2556.77it/s]


Dalam praktiknya, kamu tidak dapat memperkirakan **linear regression** menggunakan gradient descent

Untuk mendapatkan koeffisien yang tepat menggunakan **linear algebra** teknik bahwa :

In [43]:
import numpy as np

def least_square_approximation(vector_x, vector_y):
    """
    Menghitung beta dengan menggunakan numpy untuk least squares approximation.
    """
    X = np.array(vector_x)
    y = np.array(vector_y)
    return np.linalg.lstsq(X, y, rcond=None)[0]

def least_square_approximation2(vector_x, vector_y):
    """
    Menghitung beta dengan menggunakan numpy untuk least squares approximation.
    """
    x = np.array(vector_x)
    y = np.array(vector_y)

    # A = (X^T*X)⁻¹
    transpose_x = np.transpose(x)
    temp = transpose_x @ x
    A = np.linalg.inv(temp)

    # D = X^T * y
    D = transpose_x @ y

    # β = A*D = 最終解 = (X^T*X)⁻¹ * X^T * y
    β = A @ D
    return list(np.round(β,3))

In [44]:
least_square_approximation(inputs, daily_minutes_good)

array([30.57901812,  0.97250518, -1.86503639,  0.9232007 ])

In [45]:
least_square_approximation2(inputs, daily_minutes_good)
# Output:

[np.float64(30.579), np.float64(0.973), np.float64(-1.865), np.float64(0.923)]

## Setelah dihitung, persamaannya adalah:
```
menit = 30,579 + 0,973 teman - 1,865 jam kerja + 0,923 gelar doktor
```

# Model Penjelasan

**Jika faktor lainnya sama**:
* Setiap teman tambahan meningkatkan waktu yang dihabiskan di situs hampir 1 menit per hari.
* Setiap jam tambahan waktu kerja yang dihabiskan oleh pengguna mengurangi waktu yang dihabiskan di situs hampir 2 menit per hari.
* Setiap pengguna dengan gelar PhD menghabiskan hampir 1 menit lebih banyak di situs per hari.

Interaksi antara variabel-variabel ini mungkin memiliki efek yang berbeda pada waktu kerja bagi orang-orang dengan lebih banyak atau lebih sedikit teman.

# Kesesuaian
Gunakan nilai **R-kuadrat** untuk mengamati:

In [46]:
def mean(xs: List[float]) -> float:
    """Menghitung rata-rata dari daftar angka."""
    return sum(xs) / len(xs)

def de_mean(xs: List[float]) -> List[float]:
    """Menghitung de-mean dari daftar angka."""
    x_bar = mean(xs)
    return [x - x_bar for x in xs]

In [49]:
def sum_of_squares(x: List[float], y: List[float], beta: Vector) -> float:
    """Menghitung jumlah kuadrat dari error."""
    return sum(error(x_i, y_i, beta) ** 2 for x_i, y_i in zip(x, y))

def total_sum_squares(y: List[float]) -> float:
    """Menghitung total sum of squares dari y."""
    return sum(v ** 2 for v in de_mean(y))

def multiple_r_squared(xs: List[float], ys: Vector, beta: Vector) -> float:
    """
    Menghitung R-squared untuk model regresi linear berganda.
    Ini mengukur proporsi variansi dalam y yang dapat dijelaskan oleh x.
    """
    sum_of_squared_errors = sum(error(x, y, beta) ** 2 for x, y in zip(xs, ys))
    return 1.0 - (sum_of_squared_errors / total_sum_squares(ys))

def total_sum_of_squares(y: List[float]) -> float:
    """Menghitung total sum of squares dari y."""
    return sum(v ** 2 for v in de_mean(y))

In [52]:
rsq = multiple_r_squared(inputs, daily_minutes_good, beta)
assert 0.67 < rsq < 0.68  # R-squared value should be around 0.67
print("R-squared: %.3f" %rsq)  # Output: R-squared: 0.67

R-squared: 0.680


Dalam model **multiple regression** kita perlu mengapati simpangan baku setiap koefisien untuk mengukur setiap estimasi bi yang diperoleh.

**Asumsi untuk mengukur galat**:
Istilah galat εi adalah **variabel acak normal independen** dengan rata-rata 0 dan simpangan baku σ.
Semakin besar simpangan baku koefisien dalam model, semakin rendah keandalan koefisien.

# Bootstrap (pengambilan sampel berulang)
Misalkan kita memiliki sekumpulan sampel yang berisi N data, yang dihasilkan menurut beberapa distribusi yang tidak diketahui:

```
data = get_sample(num_points = n
```
Jika kita berulang kali memperoleh beberapa kelompok sampel, kita dapat menghitung median dari banyak kelompok sampel dan kemudian mengamati distribusi median ini.
Dalam kasus ini, "pengambilan sampel berulang (bootstrap)" dapat digunakan untuk memilih kembali n data dari kumpulan data asli dan menyusunnya kembali menjadi kumpulan data baru untuk menggantikan yang asli.

In [60]:
from typing import TypeVar, Callable

X = TypeVar('X')
Stat = TypeVar('Stat')

def bootstrap_sample(data: List[X]) -> List[X]:
    """Mengambil sampel bootstrap dari data."""
    return [random.choice(data) for _ in data]

def bootstrap_statistic(data: List[X], stats_fn: Callable[[List[X]], Stat], num_samples: int) -> List[Stat]:
    """Menghitung statistik dari sampel bootstrap."""
    return [stats_fn(bootstrap_sample(data)) for _ in range(num_samples)]

Sebagai contoh, menggunakan dua dataset

In [65]:
# 101 points all very close to 100
close_to_100 = [99.5 + random.random() for _ in range(101)]

# 101 points, 50 dari mereka dekat 0, 50 dari mereka dekat 200
far_from_100 = ([99.5 + random.random()] + [random.random() for _ in range(50)] + [200 + random.random() for _ in range(50)])

Mengitung median dari dua dataset, keduanya mendekati ke 100, namun terlikat seperti:

In [66]:
def sum_of_squares(v: List[float]) -> float:
    """Mengiting sum of squares dari vektor."""
    return dot(v,v)

def _median_odd(xs: List[float]) -> float:
    """Menghitung median dari daftar angka dengan panjang ganjil."""
    return sorted(xs)[len(xs) // 2]

def _median_even(xs: List[float]) -> float:
    """Menghitung median dari daftar angka dengan panjang genap."""
    sorted_xs = sorted(xs)
    mid = len(xs) // 2
    return (sorted_xs[mid - 1] + sorted_xs[mid]) / 2

def median(v: List[float]) -> float:
    """Menghitung median dari daftar angka."""
    return _median_even(v) if len(v) % 2 == 0 else _median_odd(v)

import math

def variance(xs: List[float]) -> float:
    """Menghitung variansi dari daftar angka."""
    assert len(xs), "Daftar tidak boleh kosong"

    n = len(xs)
    deviations = de_mean(xs)
    return sum_of_squares(deviations) / (n - 1)

def standard_deviation(xs: List[float]) -> float:
    """Menghitung deviasi standar dari daftar angka."""
    return math.sqrt(variance(xs))

In [67]:
median_close = bootstrap_statistic(close_to_100, median, 100)
median_far = bootstrap_statistic(far_from_100, median, 100)

assert standard_deviation(median_close) < 1
assert standard_deviation(median_far) > 90

print("Standard deviation of median for close to 100:", standard_deviation(median_close))
print("Standard deviation of median for far from 100:", standard_deviation(median_far))

Standard deviation of median for close to 100: 0.06420190628801245
Standard deviation of median for far from 100: 92.87548957205084


### Standard Error of Regression Coefficients

Kita dapat menggunakan pendekatan yang sama memperkirakan **standard error** regression koefisien.

In [69]:
from typing import Tuple

import datetime

def estimate_sample_beta(pairs: List[Tuple[Vector, float]]):
    x_sample = [x for x , _ in pairs]
    y_sample = [y for _, y in pairs]
    beta = least_squares_fit(x_sample, y_sample, learning_rate, 5000, 25)
    print("bootsrap_sample", beta)
    return beta

random.seed(0) # so that we can reproduce the results

bootstrap_betas = bootstrap_statistic(
    list(zip(inputs, daily_minutes_good)),
    estimate_sample_beta,
    100
)

least squares fit: 100%|██████████| 5000/5000 [00:01<00:00, 2584.67it/s]


bootsrap_sample [30.49402029547432, 1.0393791030498776, -1.9516851948558502, 0.7483721251697333]


least squares fit: 100%|██████████| 5000/5000 [00:02<00:00, 2424.48it/s]


bootsrap_sample [30.149963287526045, 1.0005300432763113, -2.0650380122822543, 3.177179854834797]


least squares fit: 100%|██████████| 5000/5000 [00:02<00:00, 2234.14it/s]


...

bootsrap_sample [31.725525991297584, 0.8939775539447468, -1.843559060469148, -0.6224324630864045]


least squares fit: 100%|██████████| 5000/5000 [00:02<00:00, 2438.36it/s]


bootsrap_sample [30.2731194689144, 0.8005769229528958, -1.6991234036996576, 0.9748341305369917]


least squares fit: 100%|██████████| 5000/5000 [00:02<00:00, 2387.47it/s]

bootsrap_sample [31.75696506368952, 1.0790800487199688, -2.0880894078200054, 1.6420943383461737]


In [72]:
bootstrap_standard_errors = [
    standard_deviation([beta[i] for beta in bootstrap_betas]) for i in range(len(beta))
    for i in range(4)
]
print(bootstrap_standard_errors)

# [1.272, # constant term actual error = 1.19
# 0.103 # num_friends actual error = 0.080
# 0.155 # work_hours actual error = 0.127
# 1.249] # has_phd actual error = 0.998

[1.2715078186272781, 0.10318410116073963, 0.15510591689663628, 1.2490975248051257, 1.2715078186272781, 0.10318410116073963, 0.15510591689663628, 1.2490975248051257, 1.2715078186272781, 0.10318410116073963, 0.15510591689663628, 1.2490975248051257, 1.2715078186272781, 0.10318410116073963, 0.15510591689663628, 1.2490975248051257]


Kita mungkin akan mendapatkan perkiraan baik jika kita mendapatkan lebih dari 1000 sampel menggunakan 5000 iterasi untuk perkiraan setiap beta, tapi kita tidak punya waktu seharian

Kita dapat menggunakan test hipotesis apakah koefisien bi sama dengan nol.
Dan mengusulkan hipotesis berikut:
* Hipotesis nol (H0): **Bi=0**
* Hipotesis alternatif(H1): **Bi/=0**



In [79]:
import math

def normal_cdf(x: float, mu: float = 0, sigma: float = 1) -> float:
    """Menghitung cumulative distribution function (CDF) dari distribusi normal."""
    return (1 + math.erf((x - mu) / math.sqrt(2) / sigma)) / 2

In [80]:
def p_value(beta_hat_j: float, sigma_hat_j: float) -> float:
    """
    Menghitung p-value untuk koefisien beta.
    Menggunakan distribusi normal standar.
    """
    if sigma_hat_j > 0:
        # if the coefficient is positive, we need to compute twice the
        # probability of seeing an even *larger* value
        return 2 * (1 - normal_cdf(beta_hat_j / sigma_hat_j))
    else:
        # otherwise twice the probability of seeing a *smaller* value
        return 2 * normal_cdf(beta_hat_j / sigma_hat_j)


In [88]:
assert p_value(30.58, 1.27) < 0.001 # constant term
assert p_value(0.972, 0.103) < 0.001 # num_friends
# assert p_value(-1.865, 0.155) < 0.001 # work_hours
print("p-value for work_hours:", p_value(-1.865, 0.155))
# assert p_value(0.923, 1.249) < 0.4 # has_phd
print("p-value for has_phd:", p_value(0.923, 1.249))

p-value for work_hours: 2.0
p-value for has_phd: 0.4599123452566003


### Regularization

In [85]:
# alpha is a *hyperparameter* controlling how harsh the penalty is.
# Sometimes it's called "lambda" but that already means something in Python

def ridge_penalty(beta: Vector, alpha: float) -> float:
    """
    Menghitung penalti ridge untuk vektor beta.
    Penalti ridge adalah alpha * ||beta||^2
    """
    return alpha * sum(v ** 2 for v in beta)

def squared_error_ridge(x: Vector, y: float, beta: Vector, alpha: float) -> float:
    """Estimate error plus rigde penalty on beta"""
    return error(x, y, beta) ** 2 + ridge_penalty(beta, alpha)

Lalu masukan ke dalam penurunan gradien dengan cara biasa:

In [86]:
def ridge_penalty_gradient(beta: Vector, alpha: float) -> Vector:
    """gradient of just the ridge penalty"""
    return [0.] + [2 * alpha * v for v in beta[1:]]  # skip the first element (bias term)

def sqerror_ridge_gradient(x: Vector, y: float, beta: Vector, alpha: float) -> Vector:
    """
    the gradient sesuai untuk i-th squared error term termasuk the rigde penalty
    """

    return add(
        sqerror_gradient(x, y, beta),
        ridge_penalty_gradient(beta, alpha)
    )

In [96]:
def least_squares_fit_ridge(
          xs:List[Vector],
          ys:List[float],
          alpha:float=0.0,
          learning_rate:float=0.001,
          num_steps:int=1000,
          batch_size:int=1)->Vector:

  guess = [random.random() for _ in xs[0]]

  for _ in tqdm.trange(num_steps,desc="least squares fit"):
    for start in range(0,len(xs),batch_size):
      batch_xs = xs[start:start+batch_size]
      batch_ys = ys[start:start+batch_size]

      gradient = vector_mean([sqerror_ridge_gradient(x,y,guess,alpha) for x,y in zip(batch_xs,batch_ys)])

      guess = gradient_step(guess,gradient,-learning_rate)

  return guess

Dan selanjutnya kita hanya butuh untuk memodifikasi the `least_squares_fit` fungsi untuk menggunakan `sqerror_ridge_gradient` alih-alih `sqerror_gradient`

In [98]:
# Set alpha to 0, there's no penalty at all dan kita mendapatkan hasil yang sama seperti sebelumnya.

random.seed(0)  # so that we can reproduce the results

learning_rate=0.001
beta_0 = least_squares_fit_ridge(inputs, daily_minutes_good, 0.0, learning_rate, 5000, 25)

# [30.51, 0.97, -1.85, 0.92]
assert 5 < dot(beta_0[1:], beta_0[1:]) < 6  # sum of squares of beta_0[1:] should be around 5.5
assert 0.67 , multiple_r_squared(inputs, daily_minutes_good, beta_0) < 0.69 # R-squared should be around 0.68
print("beta_0:", list(np.round(beta_0, 2)))
# Now let's try with a small alpha, say 0.01
print(dot(beta_0[1:], beta_0[1:]))  # sum of squares of beta_0[1:] should be around 5.5
print(multiple_r_squared(inputs, daily_minutes_good, beta_0))  # R-squared should be around 0.68

least squares fit: 100%|██████████| 5000/5000 [00:03<00:00, 1545.34it/s]

beta_0: [np.float64(30.51), np.float64(0.97), np.float64(-1.85), np.float64(0.91)]
5.21088501552135
0.6799849346187969


### Kita meningkatkan alpha
Kecocokan akan semakin memburuk tetapi nilai beta menjadi semakin kecil

In [102]:
random.seed(0)
learning_rate=0.001
beta_0_1 = least_squares_fit_ridge(inputs,daily_minutes_good,0.1,learning_rate,5000,25)

#[30.8,0.95,-1.83,0.54]
print("\nbeta_0_1:",list(np.round(np.array(beta_0_1),2)))
print("beta_0_1內積：",dot(beta_0_1[1:],beta_0_1[1:]) )
assert 4 < dot(beta_0_1[1:],beta_0_1[1:]) < 5
assert 0.67 < multiple_r_squared(inputs,daily_minutes_good,beta_0_1) < 0.69

least squares fit: 100%|██████████| 5000/5000 [00:03<00:00, 1586.29it/s]


beta_0_1: [np.float64(30.8), np.float64(0.95), np.float64(-1.83), np.float64(0.54)]
beta_0_1內積： 4.554201608410774


In [103]:
random.seed(0)
learning_rate=0.001
beta_1 = least_squares_fit_ridge(inputs,daily_minutes_good,1.0,learning_rate,5000,25)

#[30.65,0.9,-1.68,0.1]
print("\nbeta_1:",list(np.round(np.array(beta_1),2)))
print("beta_1內積：",dot(beta_1[1:],beta_1[1:]) )
assert 3 < dot(beta_1[1:],beta_1[1:]) < 4
assert 0.67 < multiple_r_squared(inputs,daily_minutes_good,beta_1) < 0.69

least squares fit: 100%|██████████| 5000/5000 [00:03<00:00, 1527.03it/s]


beta_1: [np.float64(30.65), np.float64(0.9), np.float64(-1.68), np.float64(0.1)]
beta_1內積： 3.6274483410516174


In [104]:
random.seed(0)
learning_rate=0.001
beta_10 = least_squares_fit_ridge(inputs,daily_minutes_good,10,learning_rate,5000,25)

#[28.31, 0.67, -0.9, -0.01]
print("\nbeta_10:",list(np.round(np.array(beta_10),2)))
print("beta_10內積：",dot(beta_10[1:],beta_10[1:]) )
assert 1 < dot(beta_10[1:],beta_10[1:]) < 2
assert 0.5 < multiple_r_squared(inputs,daily_minutes_good,beta_10) < 0.6

least squares fit: 100%|██████████| 5000/5000 [00:03<00:00, 1593.86it/s]


beta_10: [np.float64(28.31), np.float64(0.67), np.float64(-0.9), np.float64(-0.01)]
beta_10內積： 1.2706657437961764


## Kesimpulan
Koefisien "PhD" menghilang saat penalti meningkat, sehingga dapat disimpulkan bahwa koefisien tersebut kemungkinan besar adalah nol.
Hal ini konsisten dengan hasil estimasi sebelumnya.